# Collaborative Filtering Classification Example

> Documentação: http://spark.apache.org/docs/1.6.2/mllib-collaborative-filtering.html

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

### Carregando e traduzindo o dado

In [ ]:
data = sc.textFile("data/als_test.data")
ratings = data.map(lambda l: l.split(',')).map(lambda l: Rating( int(l[0]), int(l[1]), float(l[2]) ) )

### Construindo o modelo de recomendação usando Alternating Least Squares

In [ ]:
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

### Avaliando modelo em dados de treinamento

In [ ]:
testdata = ratings.map(lambda p: (p[0], p[1]))                                # Mapeando apenas para (usuário, item)
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))  # Predizendo a avaliações para usuários e items
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions) # Juntando as predições com as avaliações reais
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()              # Computando Erro Quadrático Médio
print "Mean Squared Error = " + str(MSE)